In [ ]:
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
from qsvt.algorithms import linear_solver
from qsvt.helper import circulant_matrix
import numpy as np

from qiskit.circuit.library import StatePreparation
from qiskit.quantum_info import Operator

import time

# matrix size exponent
N = 2
max_singval, min_singval = 0, 0

In [ ]:
def run(N, set_degree=0, fixed=False):
    A = circulant_matrix(N, fixed=fixed)
    A /= np.linalg.norm(A)

    eigvals, eigvecs = np.linalg.eig(A)

    qc = QuantumCircuit(N + 2)
    # qc0 = linear_solver(A, set_degree=-1)
    qc0 = linear_solver(A, set_degree=set_degree)
    qc.append(qc0, qargs=list(range(N + 2)))
    # qc0.draw('mpl')

    op = Operator(qc)
    mat = op.data
    A_inv = np.linalg.inv(A)
    inv_norm = np.linalg.norm(A_inv)
    A_inv /= inv_norm
    block = mat[:2**N, :2**N]
    block /= np.linalg.norm(block)

    # print(A_inv)
    # print(block)

    inv_eig_norms = [np.linalg.norm(1 / x)/inv_norm for x in eigvals]
    indices = list(range(2 ** N))

    # sort inversed eigvals according to their norm
    sorted_eig_pair = sorted(zip(inv_eig_norms, indices), reverse=True)
    inv_eig_norms, indices = zip(*sorted_eig_pair)

    inv_eig_norms = list(inv_eig_norms)
    indices = list(indices)

    # now eigvecs is sorted (decreasing)
    eigvecs = eigvecs[:, indices]

    eig_data = []
    for i in range(2 ** N):
        eigvec = eigvecs[:, i]
        exp_eig = (A_inv @ eigvec)
        qsvt_eig = (block @ eigvec)
        # print(f'exp_eig: {exp_eig}')
        # print(f'qsvt_eig: {qsvt_eig}')

        # pick one is enough (only prevent from divison by zero)
        for j, v in enumerate(eigvec):
            if not np.isclose(v, 0.):
                norm_eig = np.linalg.norm(exp_eig[j] / v)
                # print(f'norm_eig: {norm_eig}')
                qsvt_norm_eig = np.linalg.norm(qsvt_eig[j] / v)
                # print(f'qsvt_norm_eig; {qsvt_norm_eig}')

                abs_relative_err = np.abs((norm_eig - qsvt_norm_eig)/norm_eig)
                # print(f'abs relative error: {np.abs((norm_eig - qsvt_norm_eig)/norm_eig)}')

                eig_data.append(abs_relative_err)
                break

        # print(f'exp_eig norm: {np.linalg.norm(exp_eig[0])}')
        # print(f'qsvt_eig norm: {np.linalg.norm(qsvt_eig[0])}')
        # print('-------------')

    # print(eig_data)
    W, S, Vd = np.linalg.svd(A)
    global max_singval, min_singval
    max_singval = max(S)
    min_singval = min(S)
    print(f'max singular value: {max_singval}')
    print(f'min singular value: {min_singval}')

    sing_data = []
    for i in range(2 ** N):
        # w = A @ Vd[i]
        # print(W[:, i])
        # print(f'S[i]: {S[i]}')
        # print(f'Vd[i]: {Vd[i]}')
        exp_sing = (A_inv @ W[:, i])
        qsvt_sing = (block @ W[:, i])
        # print(f'exp_sing: {exp_sing}')
        # print(f'qsvt_sing: {qsvt_sing}')
        # print(f'|wi>: {W[:, i]}')
        # print(f'|vi>: {Vd[i]}')
        # print(f'calc: {Vd[i]/S[i] / inv_norm}')

        for j, v in enumerate(Vd[i]):
            if not np.isclose(v, 0.):
                abs = np.abs(exp_sing[j] / v)
                # print(f'abs: {abs}')
                qsvt_abs = np.abs(qsvt_sing[j] / v)
                # print(f'qsvt_abs; {qsvt_abs}')

                abs_relative_err = np.abs((abs - qsvt_abs)/abs)
                # print(f'abs relative error: {abs_relative_err}')

                sing_data.append(abs_relative_err)
                break

        # print('--------------------------')
    # print(sing_data)

    return (eig_data, sing_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot(N=2, T=5, fixed=False):
    Eig_data, Sing_data = [], []
    for i in range(T):
        (eig_data, sing_data) = run(N, -1, fixed)
        print(f'eig: {eig_data}')
        print(f'sing; {sing_data}')
        print('-------------')
        Eig_data.append(eig_data)
        Sing_data.append(sing_data)

    Eig_data = np.array(Eig_data)
    Sing_data = np.array(Sing_data)

    Eig_data_log = np.log10(Eig_data)
    Sing_data_log = np.log10(Sing_data)
    # Eig_data = np.array(Eig_data)
    # Sing_data = np.array(Sing_data)

    normalized_eig = (Eig_data_log - np.min(Eig_data_log)) / (np.max(Eig_data_log) - np.min(Eig_data_log))
    normalized_sing = (Sing_data_log - np.min(Sing_data_log)) / (np.max(Sing_data_log) - np.min(Sing_data_log))

    # Sample data
    # categories = ['Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']
    categories = [f'Matrix {i+1}' for i in range(T)]
    # values_comp1 = [10, 15, 8, 12, 7]
    # values_comp2 = [5, 8, 6, 10, 4]
    # values_comp3 = [8, 10, 7, 5, 9]

    # Calculate the width for each bar
    bar_width = 0.1

    # Calculate the positions of bars on the x-axis
    x_pos = np.arange(len(categories))

    ####### eigenvalue #######
    for i in range(2 ** N):
        plt.bar(x_pos + i*bar_width, normalized_eig[:, i], width=bar_width, label=f'eigenvalue {i}')

    # Add labels and title
    plt.xlabel('Matrices ($2^N * 2^N$' + f', N={N})')
    # plt.ylabel('Absolute Relative Error')
    plt.ylabel('Normalized log-absolute-relative error')
    plt.title('Relative Error (eigenvalues are ordered decreasingly) (deg=10)')
    plt.xticks(x_pos + bar_width, categories)
    plt.legend()

    # Display the chart
    plt.tight_layout()
    plt.show()

    ####### singular value #######
    for i in range(2 ** N):
        plt.bar(x_pos + i*bar_width, normalized_sing[:, i], width=bar_width, label=f'singular value {i}')

    # Add labels and title
    plt.xlabel('Matrices ($2^N * 2^N$' + f', N={N})')
    plt.ylabel('Normalized log-absolute-relative error')
    plt.title('Relative Error (singular values are ordered decreasingly) (deg=10)')
    plt.xticks(x_pos + bar_width, categories)
    plt.legend()

    # Display the chart
    plt.tight_layout()
    plt.show()

In [ ]:
for i in range(3):
    plot(i + 1, fixed=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate sample data
np.random.seed(42)
num_points = 50
x_values = np.random.rand(num_points)  # Random x values between 0 and 1
y_values = np.random.rand(num_points)  # Random y values between 0 and 1

# Create the scatter plot
plt.scatter(x_values, y_values, marker='o', color='blue', label='Data Points')

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Scatter Plot')
plt.legend()

# Display the plot
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
def plot_scatter(N=2, T=5, fixed=False):
    max_eig_err, min_sing_err = [], []
    max_singvals, min_singvals = [], []
    bad_eig_err, bad_sing_err = [], []
    for i in range(T):
        (eig_data, sing_data) = run(N, -1, fixed)
        print(f'eig: {eig_data}')
        print(f'sing; {sing_data}')
        print('-------------')

        if eig_data[0] > 1:
            bad_eig_err.append(eig_data[0]) 
            eig_data[0] = 1
        max_eig_err.append(eig_data[0])

        if sing_data[-1] > 1:
            bad_sing_err.append(sing_data[-1]) 
            sing_data[-1] = 1
        min_sing_err.append(sing_data[-1])

        max_singvals.append(max_singval)
        min_singvals.append(min_singval)

    max_eig_err = np.array(max_eig_err)
    min_sing_err = np.array(min_sing_err)


    # max sing. val / max eig. val err
    plt.scatter(max_singvals, max_eig_err, marker='o', color='blue', label='Data Points')

    # Add labels and title
    plt.xlabel('Max. singular value of $A$')
    plt.ylabel('Max. eigenvalue\'s relative error')
    plt.title('Max. singular value ($A$) / Max. eigenvalue ($A^{-1}$)')
    plt.legend()

    # Display the plot
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'./experiments/eig_singular_error/scatter/eig_n{N}_d10_1.png', format='png')
    plt.show()




    # min sing. val / max eig. val err
    plt.scatter(min_singvals, max_eig_err, marker='o', color='blue', label='Data Points')

    # Add labels and title
    plt.xlabel('Min. singular value of $A$')
    plt.ylabel('Max. eigenvalue\'s relative error')
    plt.title('Min. singular value ($A$) / Max. eigenvalue ($A^{-1}$)')
    plt.legend()

    # Display the plot
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'./experiments/eig_singular_error/scatter/eig_n{N}_d10_2.png', format='png')
    plt.show()




    # max sing. val / min sin. val err
    plt.scatter(max_singvals, min_sing_err, marker='o', color='blue', label='Data Points')

    # Add labels and title
    plt.xlabel('Max. singular value of $A$')
    plt.ylabel('Min. singular value\'s relative error')
    plt.title('Max. singular value ($A$) / Min. singular value ($A^{-1}$)')
    plt.legend()

    # Display the plot
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'./experiments/eig_singular_error/scatter/sing_n{N}_d10_1.png', format='png')
    plt.show()



    # min sing. val / min sin. val err
    plt.scatter(min_singvals, min_sing_err, marker='o', color='blue', label='Data Points')

    # Add labels and title
    plt.xlabel('Min. singular value of $A$')
    plt.ylabel('Min. singular value\'s relative error')
    plt.title('Min. singular value ($A$) / Min. singular value ($A^{-1}$)')
    plt.legend()

    # Display the plot
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'./experiments/eig_singular_error/scatter/sing_n{N}_d10_2.png', format='png')
    plt.show()

    return (bad_eig_err, bad_sing_err)

In [ ]:
for i in range(3):
    plot_scatter(N=i+1, T=100)

### 結論

- Eigenvalue (${1\over \lambda}$) 越大保持越好
- Singular value (${1\over \sigma}$) 越小保持越好
  - 符合預期！因為 ${1\over \sigma}$ 小代表 ${\sigma}$ 大！